In [1]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin
from catboost import CatBoostClassifier

import lightgbm as lgb
import joblib

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

In [3]:
state_dict = joblib.load('/kaggle/input/fitted-models/week_num_dict.pkl')
cat_cols = state_dict['cat_cols']
train_cols = state_dict['train_cols']
lgb = state_dict['lgb'][5:]
cbc = state_dict['cat'][5:]

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
#         for col in df.columns:
#             if col not in ["target", "case_id", "WEEK_NUM"]:
#                 isnull = df[col].is_null().mean()
#                 if isnull > 0.7:
#                     df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df


class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return expr_max +expr_last+expr_mean
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return  expr_max +expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
# data_store = {
#     "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
#     "depth_0": [
#         read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
#         read_files(TRAIN_DIR / "train_static_0_*.parquet"),
#     ],
#     "depth_1": [
#         read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
#         read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
#         read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
#         read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_other_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_person_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
#         read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
#     ],
#     "depth_2": [
#         read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
#         read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
#         read_files(TRAIN_DIR / "train_person_2.parquet", 2),
#         read_files(TRAIN_DIR / "train_applprev_2.parquet", 2),
#     ]
# }

In [6]:
# df_train = feature_eng(**data_store)
# print("train data shape:\t", df_train.shape)
# del data_store
# gc.collect()
# df_train = df_train.pipe(Pipeline.filter_cols)
# df_train, cat_cols = to_pandas(df_train)
# df_train = reduce_mem_usage(df_train)
# print("train data shape:\t", df_train.shape)
# nums=df_train.select_dtypes(exclude='category').columns
# from itertools import combinations, permutations
# #df_train=df_train[nums]
# nans_df = df_train[nums].isna()
# nans_groups={}
# for col in nums:
#     cur_group = nans_df[col].sum()
#     try:
#         nans_groups[cur_group].append(col)
#     except:
#         nans_groups[cur_group]=[col]
# del nans_df; x=gc.collect()

# def reduce_group(grps):
#     use = []
#     for g in grps:
#         mx = 0; vx = g[0]
#         for gg in g:
#             n = df_train[gg].nunique()
#             if n>mx:
#                 mx = n
#                 vx = gg
#             #print(str(gg)+'-'+str(n),', ',end='')
#         use.append(vx)
#     return use

# def group_columns_by_correlation(matrix, threshold=0.8):
#     # 计算列之间的相关性
#     correlation_matrix = matrix.corr()

#     # 分组列
#     groups = []
#     remaining_cols = list(matrix.columns)
#     while remaining_cols:
#         col = remaining_cols.pop(0)
#         group = [col]
#         correlated_cols = [col]
#         for c in remaining_cols:
#             if correlation_matrix.loc[col, c] >= threshold:
#                 group.append(c)
#                 correlated_cols.append(c)
#         groups.append(group)
#         remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
#     return groups

# uses=[]
# for k,v in nans_groups.items():
#     if len(v)>1:
#             Vs = nans_groups[k]
#             grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
#             use=reduce_group(grps)
#             uses=uses+use
#     else:
#         uses=uses+v
# #     print('####### NAN count =',k)
# uses=uses+list(df_train.select_dtypes(include='category').columns)
# df_train=df_train[uses]

In [7]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_files(TEST_DIR / "test_person_2.parquet", 2),
        read_files(TEST_DIR / "test_applprev_2.parquet", 2),
    ]
}

In [8]:
df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)
del data_store
gc.collect()
# df_test = df_test.select([col for col in df_train.columns if col != "target"])
# print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

# cat_cols = [col for col in cat_cols if col in df_test.columns]
df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

gc.collect()

test data shape:	 (10, 860)
test data shape:	 (10, 860)
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.05 MB
Decreased by 33.4%


0

In [9]:
df_test = df_test[train_cols + ['case_id']]
df_test[cat_cols] = df_test[cat_cols].astype('str')

df_test.shape, len(cat_cols)

((10, 400), 79)

In [10]:
class VotingModel():
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:]]
        
        return np.mean(y_preds, axis=0)

In [11]:
model = VotingModel(cbc+lgb)

In [12]:
df_test = df_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(df_test)[:,1], index=df_test.index)
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred

df_subm.to_csv("submission.csv")
df_subm

,score
case_id,
57543,0.008524
57549,0.041243
57551,0.002945
57552,0.013109
57569,0.096083
57630,0.009015
57631,0.021138
57632,0.006781
57633,0.031769


In [13]:
print("Check null: ", df_subm["score"].isnull().any())

df_subm.head()

Check null:  False


,score
case_id,
57543,0.008524
57549,0.041243
57551,0.002945
57552,0.013109
57569,0.096083


In [14]:
df_subm.to_csv("submission.csv")